In [41]:
import json
import boto3
import tweepy

In [42]:
ssm_client = boto3.client('ssm')
firehose_client = boto3.client('firehose')

In [43]:
consumer_key = ssm_client.get_parameter(
    Name='/twitter-api/consumer-key',
    WithDecryption=True
)['Parameter']['Value']

consumer_secret = ssm_client.get_parameter(
    Name='/twitter-api/consumer-secret',
    WithDecryption=True
)['Parameter']['Value']

In [44]:
access_token = ssm_client.get_parameter(
    Name='/twitter-api/access-token',
    WithDecryption=True
)['Parameter']['Value']

access_token_secret = ssm_client.get_parameter(
    Name='/twitter-api/access-token-secret',
    WithDecryption=True
)['Parameter']['Value']

In [45]:
delivery_stream_name = 'nept-mlops-dev-tweet-delivery-stream'

In [46]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [50]:
class TweetProducer(tweepy.StreamListener):
    def __init__(self, firehose_client):
        self.firehose_client = firehose_client

    def on_data(self, data):
        tweet = json.loads(data)
        self.firehose_client.put_record(
            DeliveryStreamName=delivery_stream_name, 
            Record={
                'Data':json.dumps(tweet).encode('utf-8')
            }
        )
        print(tweet)
        return True
    
    def on_error(self, status):
        print("Error: " + str(status))

In [ ]:
tweet_producer = TweetProducer(firehose_client)
stream = tweepy.Stream(auth=auth, listener=tweet_producer)
stream.filter(track=['#aws'])